In [2]:
import pandas as pd
import numpy as np
import os 
import geopandas as gpd
import plotly.express as px
from scipy.spatial import KDTree
import plotly.graph_objects as go
from plotly.subplots import make_subplots

mf = gpd.read_file('./Marine_forest/Marine_forest.shp')
count = len(mf[(mf['면적'] == mf['구역']) & (mf['면적'] == mf['규모'])])
mf.drop(columns=['구역','규모'],inplace=True)
col_names = {'시도명': 'province','시군구명':'city','시군구코드':'city_code','위도':'latitude','경도':'longitude','마을어장':'town','면적':'area'}
mf.rename(columns=col_names, inplace=True)
#기본 칼럼인 위도 경도는 오류가 있었음
mf['latitude'] = mf['geometry'].y
mf['longitude'] = mf['geometry'].x

상관관계 분석 및 관련 논문을 참조한 결과,

1) 물리적 원인
가장 유력한 원인은 수온의 상승과 이로 인한 바닷속의 용존산소량의 감소
이 밖에도 다시마와 같은 해조류를 섭취하는 조식동물의 지나친 증식과 오염된 담수의 유입

2) 화학적 원인
해수의 저염수화(바다에 담수가 많이 유입되어 해수의 염도가 낮아짐)와 
해양오염(불투명도 증가와 오염물질의 확산)

따라서 
저희가 가지고 있는 데이터셋에서는
1. 수온, 2. 용존산소, 3. 염분,  4. 부유물질 농도 순으로 보면 될 것 같습니다!

칼럼별 상관계수 측정

In [24]:
df = pd.read_csv('./data/merged_data.csv')

df.drop(columns=['station', 'date_time', 'longitude', 'latitude','bot.depth[m]','depth(text)','depth(m)','전체수심(m)','유분[mg/L]','용존무기질소[μg/L]','아질산성질소[μg/L]', '인산염인[μg/L]',
       '질산성질소[μg/L]','암모니아성 질소[μg/L]'],inplace=True)
# 상관계수 계산
print(df.columns)
correlation_matrix = df.corr()
title = "Pearson Correlation Matrix"
fig = px.imshow(correlation_matrix, aspect="auto",title=title)
fig.update_layout(width=600, height=600, margin=dict(t=50, b=50, l=50, r=50))
fig.show()


Index(['총질소[μg/L]', '염분[psu]', '수온[℃]', '클로로필-a[μg/L]', '부유물질 농도[μg/L]',
       '용존산소[mg/L]', '수소이온농도[무단위]', '규산염[μg/L]', '투명도[m]', '화학적산소요구량[mg/L]',
       '총인[μg/L]', '부유물질 농도[mg/L]'],
      dtype='object')


In [25]:
spearman_corr = df.corr(method='kendall')
title = "Kendall Correlation Matrix"
fig = px.imshow(correlation_matrix, aspect="auto",title=title)
fig.update_layout(width=600, height=600, margin=dict(t=50, b=50, l=50, r=50))
fig.show()


In [23]:
spearman_corr = df.corr(method='spearman')
title = "Spearman Correlation Matrix"
fig = px.imshow(correlation_matrix, aspect="auto",title=title)
fig.update_layout(width=600, height=600, margin=dict(t=50, b=50, l=50, r=50))
fig.show()


# 바다숲 지형정보

바다숲 전체

In [25]:
title = '2013~2022년 바다숲 조성사업 현황'
buff = mf.loc[mf['시설년도'] <= 2022]
buff = buff.to_crs(epsg=32652)
scaling_factor = 1
center = {'lat': 35.8, 'lon': buff['longitude'].median()}
fig = px.scatter_mapbox(buff, lat=buff['latitude'], lon=buff['longitude'], size=(buff['area']* 10000)*scaling_factor,
                        color=buff['area'], size_max=20, zoom=5, center=center, title =title)
fig.update_layout(mapbox_style="open-street-map", margin={"r":10,"t":70,"l":20,"b":20},
                  width=1000 , height=800,legend=dict(orientation="h",yanchor="bottom",y=1.02,xanchor="right",x=1.15))
fig.show()

특정 년도마다 해양지형정보 측정기와 바다숲 위치정보

In [24]:
def map_scatter(title, ocean_loc):
    title = f'{title}'
    ocean_df  = pd.read_csv(f'{ocean_loc}')

    buff = mf.loc[mf['시설년도'] <= int(ocean_loc[-8:-4])]
    buff = buff.to_crs(epsg=32652)
    scaling_factor = 1
    center = {'lat': 35.8, 'lon': buff['longitude'].median()}
    fig = px.scatter_mapbox(buff, lat=buff['latitude'], lon=buff['longitude'], size=(buff['area']* 10000)*scaling_factor,
                            color=buff['area'], size_max=30, zoom=5, center=center, title =title)
    fig.add_trace(
        go.Scattermapbox(
            lat=ocean_df['latitude'],
            lon=ocean_df['longitude'],
            mode='markers',
            marker=dict(size=5, color='red'),  # 예시로 빨간색 마커를 사용
            
            name='Surface Data',
            
        )
    )
    fig.update_layout(mapbox_style="open-street-map", margin={"r":10,"t":70,"l":20,"b":20},
                    width=800 , height=800,legend=dict(orientation="h",yanchor="bottom",y=1.02,xanchor="right",x=1.15))

    fig.show()


def sc_by_col(ocean_path, origin_path, radius):
    columns_to_plot = ['총질소[μg/L]', '염분[psu]','수온[℃]', '클로로필-a[μg/L]', 
                    '용존산소[mg/L]', '수소이온농도[무단위]','규산염[μg/L]', 
                    '아질산성질소[μg/L]', '인산염인[μg/L]', '질산성질소[μg/L]', '투명도[m]','용존무기질소[μg/L]', 
                    '화학적산소요구량[mg/L]', '암모니아성 질소[μg/L]', '총인[μg/L]']
    # 데이터 불러오기
    ocean_df = pd.read_csv(ocean_path)
    origin = pd.read_csv(origin_path)

    # date_time 컬럼을 연-월 형식으로 변경
    ocean_df['date_time'] = pd.to_datetime(ocean_df['date_time']).dt.strftime('%Y-%m')
    origin['date_time'] = pd.to_datetime(origin['date_time']).dt.strftime('%Y-%m')


    # 각 데이터프레임에서 date_time을 기준으로 평균 계산
    ocean_avg = ocean_df.groupby('date_time')[columns_to_plot].mean().reset_index()
    origin_avg = origin.groupby('date_time')[columns_to_plot].mean().reset_index()

    # 시각화
    fig = make_subplots(rows=len(columns_to_plot), cols=1, shared_xaxes=True)

    for i, col in enumerate(columns_to_plot):
        fig.add_trace(go.Scatter(x=ocean_avg['date_time'], y=ocean_avg[col], mode="lines", name=f"반경{radius}km내의 정보 {col}"), row=i+1, col=1)
        fig.add_trace(go.Scatter(x=origin_avg['date_time'], y=origin_avg[col], mode="lines", name=f"전체 {col}"), row=i+1, col=1)

    fig.update_layout(width=1200, height=200*len(columns_to_plot), 
                    margin=dict(t=50, b=50, l=50, r=50),
                    legend=dict(
                        x=1.05,  # 범례의 x 위치 조절
                        y=0.5,   # 범례의 y 위치 조절
                        itemsizing='constant' # 범례 항목 간의 간격 조절
                    ))

    fig.show()

2020, 반경이 555m내의 측정기와 바다숲의 정보

In [26]:
map_scatter("2018년도 해양지형정보, 바다숲 위치(0.555km)",'data/surface/0.005/2018.csv')

In [21]:
ocean_df = pd.read_csv(f'data/surface/0.01/2018.csv')
print(ocean_df.columns)
unique_values = ocean_df['station'].unique().tolist()
print(unique_values)
fig = make_subplots(rows=4, cols=2)

columns_to_plot = ['총질소[μg/L]', '총인[μg/L]', '용존무기질소[μg/L]','화학적산소요구량[mg/L]','암모니아성 질소[μg/L]']
for i, d in enumerate(unique_values):
    row = i // 2 + 1  # 행 결정
    col = i % 2 + 1   # 열 결정
    x = ocean_df.loc[ocean_df['station'] == d].sort_values(by='date_time').reset_index()
    fig.add_trace(go.Scatter(x=x['date_time'], y=x['용존산소[mg/L]'], name=f'용존산소 {d}'), row=row, col=col)
    fig.add_trace(go.Scatter(x=x['date_time'], y=x['총질소[μg/L]'], name=f'총질소 {d}'), row=row, col=col)
    fig.add_trace(go.Scatter(x=x['date_time'], y=x['총인[μg/L]'], name=f'총인 {d}'), row=row, col=col)
    fig.add_trace(go.Scatter(x=x['date_time'], y=x['용존무기질소[μg/L]'], name=f'용존무기질소 {d}'), row=row, col=col)
    fig.add_trace(go.Scatter(x=x['date_time'], y=x['화학적산소요구량[mg/L]'], name=f'화학적산소요구량 {d}'), row=row, col=col)


    
fig.update_layout(width=1200, height=1200, margin=dict(t=50, b=50, l=50, r=50))

fig.show()

Index(['station', 'date_time', 'longitude', 'latitude', '총질소[μg/L]', '염분[psu]',
       '수온[℃]', '클로로필-a[μg/L]', '부유물질 농도[μg/L]', '용존산소[mg/L]', '수소이온농도[무단위]',
       '규산염[μg/L]', '아질산성질소[μg/L]', '인산염인[μg/L]', '질산성질소[μg/L]', '투명도[m]',
       '용존무기질소[μg/L]', '화학적산소요구량[mg/L]', '암모니아성 질소[μg/L]', '총인[μg/L]',
       'overlap_count', '유분[mg/L]', '부유물질 농도[mg/L]'],
      dtype='object')
['가로림연안4_연안', '기장연안4_연안', '서귀포연안1_연안', '속초연안1_연안', '속초연안2_연안', '거제도동안2_연안', '거진연안2_연안', '거진연안H1_항만']


5km

In [52]:
map_scatter("2018년도 해양지형정보, 바다숲 위치(5.555km)",'data/surface/0.05/2018.csv')

In [60]:
ocean_df = pd.read_csv(f'data/surface/0.05/2018.csv')
origin = pd.read_csv('./data/2018.csv')
print(ocean_df.columns)
print(len(origin['station'].unique().tolist()))
unique_values = ocean_df['station'].unique().tolist()
print("개수:",len(unique_values))
fig = make_subplots(rows=5, cols=2)


for i, d in enumerate(unique_values[:10]):
    row = i // 2 + 1  # 행 결정
    col = i % 2 + 1   # 열 결정
    x = ocean_df.loc[ocean_df['station'] == d].sort_values(by='date_time').reset_index()
    
    fig.add_trace(go.Scatter(x=x['date_time'], y=x['용존산소[mg/L]'], name=f'용존산소 {d}'), row=row, col=col)
    fig.add_trace(go.Scatter(x=x['date_time'], y=x['총질소[μg/L]'], name=f'총질소 {d}'), row=row, col=col)
    fig.add_trace(go.Scatter(x=x['date_time'], y=x['총인[μg/L]'], name=f'총인 {d}'), row=row, col=col)
    fig.add_trace(go.Scatter(x=x['date_time'], y=x['용존무기질소[μg/L]'], name=f'용존무기질소 {d}'), row=row, col=col)
    fig.add_trace(go.Scatter(x=x['date_time'], y=x['화학적산소요구량[mg/L]'], name=f'화학적산소요구량 {d}'), row=row, col=col)
    fig.add_trace(go.Scatter(x=x['date_time'], y=x['염분[psu]'], name=f'화학적산소요구량 {d}'), row=row, col=col)
    fig.add_trace(go.Scatter(x=x['date_time'], y=x['아질산성질소[μg/L]'], name=f'아질산성질소 {d}'), row=row, col=col)

fig.update_layout(width=1200, height=1200, margin=dict(t=50, b=50, l=50, r=50))

fig.show()

Index(['station', 'date_time', 'longitude', 'latitude', '총질소[μg/L]', '염분[psu]',
       '수온[℃]', '클로로필-a[μg/L]', '부유물질 농도[μg/L]', '용존산소[mg/L]', '수소이온농도[무단위]',
       '규산염[μg/L]', '아질산성질소[μg/L]', '인산염인[μg/L]', '질산성질소[μg/L]', '투명도[m]',
       '용존무기질소[μg/L]', '화학적산소요구량[mg/L]', '암모니아성 질소[μg/L]', '총인[μg/L]',
       'overlap_count', '유분[mg/L]', '부유물질 농도[mg/L]'],
      dtype='object')
425
개수: 116


동해 따로 서해따로 남해 따로 해야할듯

In [25]:
sc_by_col('data/surface/0.05/2018.csv','./data/surface/merged/2018.csv',5)

Exception: The (row, col) pair sent is out of range. Use Figure.print_grid to view the subplot grid. 

1km

In [5]:
map_scatter("2018년도 해양지형정보, 바다숲 위치(1km)",'data/surface/0.01/2018.csv')

In [23]:
sc_by_col('data/surface/0.01/2018.csv','./data/surface/merged/2018.csv',1)

TypeError: sc_by_col() takes 2 positional arguments but 3 were given

## 각 station마다의 변화

In [6]:
surface_df  = pd.read_csv('./data/surface_df.csv')

unique_values = surface_df['station'].unique().tolist()
print(unique_values)


fig = make_subplots(rows=10, cols=2)

columns_to_plot = ['총질소[μg/L]', '총인[μg/L]', '용존무기질소[μg/L]','화학적산소요구량[mg/L]','암모니아성 질소[μg/L]']
for i, d in enumerate(unique_values[:20]):
    row = i // 2 + 1  # 행 결정
    col = i % 2 + 1   # 열 결정
    x = surface_df.loc[surface_df['station'] == d].reset_index()
    
    fig.add_trace(go.Scatter(x=x['date_time'], y=x['용존무기질소[μg/L]'], mode='lines+markers', name=f'Plot {d}'), row=row, col=col)

    
fig.update_layout(width=1200, height=1200, margin=dict(t=50, b=50, l=50, r=50))

fig.show()

['가로림연안1_연안', '가로림연안2_연안', '가로림연안3_연안', '가막만1_환경관리', '가막만2_환경관리', '가막만3_환경관리', '감포연안1_연안', '감포연안2_연안', '감포연안3_연안', '감포연안4_연안', '감포연안H1_항만', '강구연안1_연안', '강구연안2_연안', '강구연안H1_항만', '강릉연안1_연안', '강릉연안2_연안', '강릉연안3_연안', '강릉연안4_연안', '강릉연안5_연안', '강릉연안6_연안', '강릉연안7_연안', '거제도남안1_연안', '거제도남안2_연안', '거제도남안3_연안', '거제도남안4_연안', '거제도동안1_연안', '거제도동안2_연안', '거제도동안3_연안', '거제도동안4_연안', '거제도동안H1_항만', '거제도동안H2_항만', '거진연안1_연안', '거진연안2_연안', '거진연안H1_항만', '고성자란만1_연안', '고성자란만2_연안', '고성자란만3_연안', '고창연안1_연안', '고창연안2_연안', '고창연안3_연안', '고창연안4_연안', '고흥연안1_연안', '고흥연안2_연안', '고흥연안3_연안', '고흥연안4_연안', '고흥연안5_연안', '고흥연안6_연안', '고흥연안7_연안', '고흥연안8_연안', '광양만1_환경관리', '광양만2_환경관리', '광양만3_환경관리', '광양만4_환경관리', '광양만5_환경관리', '광양만H1_항만', '구룡포연안1_연안', '구룡포연안2_연안', '구룡포연안H1_항만', '군산연안1_연안', '군산연안10_연안', '군산연안2_연안', '군산연안3_연안', '군산연안4_연안', '군산연안5_연안', '군산연안6_연안', '군산연안7_연안', '군산연안8_연안', '군산연안9_연안', '기장연안1_연안', '기장연안2_연안', '기장연안3_연안', '기장연안4_연안', '기장연안H1_항만', '낙동강하구1_환경관리', '낙동강하구2_환경관리', '낙동강하구3_환경관리', '낙동강하구4_환경관리', '남해도남안1_연안', '남해도남안2_연안', '남

KeyError: 'date_time'

## 2010년~ 2020년 해양환경 변화 시각화

해향지형정보 한 포인트당 겹치는 바다숲의 개수와 미치는 영향에 대해 알아봄
바다숲은 통상적으로 5년이상 되야 효과가 나타난다고 본거같음

먼저 kdTree를 활용해 좌표 거리를 계산, 특정 radius에 들어온 station을 df로 만들고 각 포인트 마다 정보의 변화를 찾기